# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']


Arxiv has 69 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2402.06722


extracting tarball to tmp_2402.06722...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:473: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

✘ → 11920:\section{Observations} \label{sec:data}
  ↳ 25688:\section{Results}


✔ → 25688:\section{Results}
  ↳ 40758:\section{Discussion} \label{sec:discussion}


✔ → 40758:\section{Discussion} \label{sec:discussion}
  ↳ 61440:\section{Summary and Conclusion}\label{sec:conclusion}
✔ → 61440:\section{Summary and Conclusion}\label{sec:conclusion}
  ↳ 66437:end


E. Bañados  ->  E. Bañados  |  ['E. Bañados']


Found 72 bibliographic references in tmp_2402.06870/trystan.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.06870-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.06870) | **A lack of LAEs within 5Mpc of a luminous quasar in an overdensity at  z=6.9: potential evidence of quasar negative feedback at protocluster scales**  |
|| T. S. Lambert, et al. -- incl., <mark>E. Bañados</mark> |
|*Appeared on*| *2024-02-13*|
|*Comments*| *Submitted to A&A. 15 pages, 11 figures*|
|**Abstract**| High-redshift quasars are thought to live in the densest regions of space which should be made evident by an overdensity of galaxies around them. However, campaigns to identify these overdensities through the search of Lyman Break Galaxies (LBGs) and Lyman $\alpha$ emitters (LAEs) have had mixed results. These may be explained by either the small field of view of some of the experiments, the broad redshift ranges targeted by LBG searches, and by the inherent large uncertainty of quasar redshifts estimated from UV emission lines, which makes it difficult to place the Ly-$\alpha$ emission line within a narrowband filter. Here we present a three square degree search ($\sim 1000$ pMpc) for LAEs around the $z=6.9$ quasar VIKJ2348-3054 using the Dark Energy CAMera (DECam), housed on the 4m Blanco telescope, finding 38 LAEs. The systemic redshift of VIK J2348--3054 is known from ALMA [CII] observations and place the Ly-$\alpha$ emission line of companions within the NB964 narrowband of DECam. This is the largest field of view LAE search around a $z>6$ quasar conducted to date. We find that this field is $\sim$ 10 times more overdense when compared to the Chandra Deep-Field South, observed previously with the same instrumental setup as well as several combined blank fields. This is strong evidence that VIKJ2348-3054 resides in an overdensity of LAEs over several Mpc. Surprisingly, we find a lack of LAEs within 5 physical Mpc of the quasar and take this to most likely be evidence of the quasar suppressing star formation in its immediate vicinity. This result highlights the importance of performing overdensity searches over large areas to properly assess the density of those regions of the Universe. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.06722-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.06722) | **A $1.9\,M_{\odot}$ neutron star candidate in a 2-year orbit**  |
|| <mark>K. El-Badry</mark>, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2024-02-13*|
|*Comments*| *19 pages, 11 figures, submitted to OJAp*|
|**Abstract**| We report discovery and characterization of a main-sequence G star orbiting a dark object with mass $1.90\pm 0.04\,M_{\odot}$. The system was discovered via Gaia astrometry and has an orbital period of 731 days. We obtained multi-epoch RV follow-up over a period of 600 days, allowing us to refine the Gaia orbital solution and precisely constrain the masses of both components. The luminous star is a $\gtrsim 12$ Gyr-old, low-metallicity halo star near the main-sequence turnoff ($T_{\rm eff} \approx 6000$ K; $\log\left(g/\left[{\rm cm\,s^{-2}}\right]\right)\approx 4.0; \rm [Fe/H]\approx-1.25$; $M\approx0.79\,M_{\odot}$) with a highly enhanced lithium abundance. The RV mass function sets a minimum companion mass for an edge-on orbit of $M_2 > 1.67\,M_{\odot}$, well above the Chandrasekhar limit. The Gaia inclination constraint, $i=68.8\pm 1.4$ deg, then implies a companion mass of $M_2= 1.90\pm 0.04\,M_{\odot}$. The companion is most likely a massive neutron star: the only viable alternative is two massive white dwarfs in a close binary, but this scenario is disfavored on evolutionary grounds. The system's low eccentricity ($e=0.122\pm 0.003$) disfavors dynamical formation channels and implies that the neutron star formed with very little mass loss ($\lesssim 1\,M_{\odot}$) and with a weak natal kick ($v_{\rm kick}\lesssim 10\,\rm km\,s^{-1}$). The current orbit is too small to have accommodated the neutron star progenitor as a red supergiant or super-AGB star. The simplest formation scenario -- isolated binary evolution -- requires the system to have survived stable mass transfer or common envelope evolution with a donor-to-accretor mass ratio $>10$. The system, which we call Gaia NS1, is likely a progenitor of symbiotic X-ray binaries and long-period millisecond pulsars. Its discovery challenges binary evolution models and bodes well for Gaia's census of compact objects in wide binarie |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error expected string or bytes-like object</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2402.06870.md
    + _build/html/tmp_2402.06870/./on_sky_distribution.png
    + _build/html/tmp_2402.06870/./chiara_lambert.png
    + _build/html/tmp_2402.06870/./surface_density.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thefigure}{\arabic{figure} (Cont.)}$
$\newcommand{\thefigure}{\arabic{figure} (Cont.)}$
$\newcommand{\thefigure}{\arabic{figure}}$</div>



<div id="title">

# A lack of LAEs within 5Mpc of a luminous quasar in an overdensity at z=6.9: potential evidence of quasar negative feedback at protocluster scales

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2402.06870-b31b1b.svg)](https://arxiv.org/abs/2402.06870)<mark>Appeared on: 2024-02-13</mark> -  _Submitted to A&A. 15 pages, 11 figures_

</div>
<div id="authors">

T. S. Lambert, et al. -- incl., <mark>E. Bañados</mark>

</div>
<div id="abstract">

**Abstract:** High-redshift quasars are thought to live in the densest regions of space which should be made evident by an overdensity of galaxies around them. However, campaigns to identify these overdensities through the search of Lyman Break Galaxies (LBGs) and Lyman $\alpha$ emitters (LAEs) have had mixed results. These may be explained by either the small field of view of some of the experiments, the broad redshift ranges targeted by LBG searches, and by the inherent large uncertainty of quasar redshifts estimated from UV emission lines, which makes it difficult to place the Ly- $\alpha$ emission line within a narrowband filter. Here we present a three square degree search ( $\sim 1000$ pMpc) for LAEs around the $z=6.9$ quasar VIK J2348--3054 using the Dark Energy CAMera (DECam), housed on the 4m Blanco telescope, finding 38 LAEs. The systemic redshift of VIK J2348--3054 is known from ALMA [ CII ] observations and place the Ly- $\alpha$ emission line of companions within the NB964 narrowband of DECam. This is the largest field of view LAE search around a $z>6$ quasar conducted to date. We find that this field is $\sim$ 10 times more overdense when compared to the Chandra Deep-Field South, observed previously with the same instrumental setup as well as several combined blank fields. This is strong evidence that VIK J2348--3054 resides in an overdensity of LAEs over several Mpc. Surprisingly, we find a lack of LAEs within 5 physical Mpc of the quasar and take this to most likely be evidence of the quasar suppressing star formation in its immediate vicinity. This result highlights the importance of performing overdensity searches over large areas to properly assess the density of those regions of the Universe.

</div>

<div id="div_fig1">

<img src="tmp_2402.06870/./on_sky_distribution.png" alt="Fig7" width="100%"/>

**Figure 7. -** LAE on-sky distribution around the quasar. The black outline is the area used to identify LAE and is slightly smaller than the full DECam FOV. The star at the image center is the quasar location. The shaded region shows the proximity zone determined by [Mazzucchelli, Bañados and Venemans (2017)](). The inner and outer rings highlight 5 pMpc and 10 pMpc from the quasar respectively, with the later being the maximum scale at which objects are expected to collapse and form clusters at $z=0$ ([Overzier, Guo and Kauffmann 2009]()) . (*fig:on_sky_distribution_us*)

</div>
<div id="div_fig2">

<img src="tmp_2402.06870/./chiara_lambert.png" alt="Fig4" width="100%"/>

**Figure 4. -** Summary plot of the various studies and results of looking for LAEs and LBGs around high-redshift quasars. Squares are Lyman-break searches and diamonds are Ly-$\alpha$ studies. Filled in values indicate that an overdensity was found, whilst not filled in represents that no overdensity was found. (*fig:chiara_plot*)

</div>
<div id="div_fig3">

<img src="tmp_2402.06870/./surface_density.png" alt="Fig8" width="100%"/>

**Figure 8. -** Surface density profile as a function of distance from the central quasar. The data were binned using 2 pMpc bins. Red arrows indicate the 1 $\sigma$ upper limits based on non-detections and assuming a Poisson uncertainty. The blue shaded region is the average upper limit for the degraded CDFS data. The dashed line is the maximum theoretical collapse radius found by [Overzier, Guo and Kauffmann (2009)](). (*fig:surface density*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2402.06870"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

370  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
